In [1]:
import pandas as pd
import numpy as np
import pandas_profiling as pd_pro
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import gc
import lightgbm as lgbm
from sklearn import model_selection, preprocessing, ensemble

In [2]:
test = pd.read_csv('input/test.csv', parse_dates=['ClickDate'], 
                    dtype= { 'ID' : np.int32, 
                            'Carrier' : np.float32,                             
                             #'publisherId' : np.int32,
                            'advertiserCampaignId' : np.float32,
                            'Fraud': np.float32},
                     encoding='UTF-8')  
                   

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test.shape

(25548873, 14)

In [4]:
test.columns

Index(['ID', 'Country', 'Carrier', 'TrafficType', 'ClickDate', 'Device',
       'Browser', 'OS', 'RefererUrl', 'UserIp', 'publisherId',
       'subPublisherId', 'advertiserCampaignId', 'Fraud'],
      dtype='object')

In [5]:
test_Id = test['ID'].values

In [6]:
#chunksize = 10 ** 7
#chunks = pd.read_csv('input/train.csv', chunksize=chunksize)
train = pd.read_csv('input/train.csv', parse_dates=['ClickDate'], 
                    dtype= { 'ID' : np.int32, 
                            'Carrier' : np.float32, 
                            'ConversionPayOut' : np.float32,
                             #'publisherId' : np.int32,
                            'advertiserCampaignId' : np.float32,
                            'Fraud': np.float32},
                     encoding='UTF-8',  
                     nrows= 15000000)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
#train.columns

In [8]:
#train.shape

In [9]:
len_train = train.shape[0]
y_train = train.ConversionPayOut.values
train.drop(['ConversionStatus','ConversionDate', 'ConversionPayOut'], inplace = True, axis=1)

In [10]:
data = pd.concat([train, test])

In [11]:
del train, test
gc.collect()

7

In [12]:
data.Country.fillna('other', inplace = True)
data.TrafficType.fillna('other', inplace = True)
data.Device.fillna('other', inplace = True)
data.Browser.fillna('other', inplace = True)
data.OS.fillna('other', inplace = True)
data.RefererUrl.fillna('other', inplace = True)
data.subPublisherId.fillna('other', inplace = True)
data['publisherId'] = pd.to_numeric(data['publisherId'], errors='coerce')
data['subPublisherId'] = pd.to_numeric(data['subPublisherId'], errors='coerce')

In [13]:
cols_to_encode = ['Country',  'TrafficType', 'Device','Browser', 'OS', 'RefererUrl', 'UserIp', 
                 'subPublisherId','publisherId']
le = LabelEncoder()
for col in cols_to_encode:
    data[col] = le.fit_transform(data[col])
    

In [14]:
data['tweekday'] = data['ClickDate'].dt.weekday
data['thour'] = data['ClickDate'].dt.hour
data['tminute'] = data['ClickDate'].dt.minute

In [15]:
#data.head(5)

In [16]:
ref_url_cnt = data.groupby(['UserIp'])[ 'RefererUrl'].nunique().reset_index().rename(columns = {'RefererUrl': 'ref_url_cnt'})

In [17]:
pubId_cnt = data.groupby(['UserIp'])[ 'publisherId'].nunique().reset_index().rename(columns = {'publisherId': 'pubId_cnt'})

In [18]:
adv_comp_cnt = data.groupby(['UserIp'])[ 'advertiserCampaignId'].nunique().reset_index().\
                             rename(columns = {'advertiserCampaignId': 'compId_cnt'})

In [19]:
data = pd.merge(data, ref_url_cnt, how='left', on=['UserIp'])
data = pd.merge(data, pubId_cnt, how='left', on=['UserIp'])
data = pd.merge(data, adv_comp_cnt, how='left', on=['UserIp'])

In [20]:
del ref_url_cnt, pubId_cnt, adv_comp_cnt
gc.collect()

147

In [21]:
data['refUrl_user_cnt'] = data.groupby(['RefererUrl'])['ID'].transform('count')

In [22]:
data['user_cnt'] = data.groupby(['UserIp'])['ID'].transform('count')

In [23]:
data['user_week_cnt'] = data.groupby(['UserIp', 'tweekday'])['ID'].transform('count')
data['user_hr_cnt'] = data.groupby(['UserIp', 'tweekday', 'thour'])['ID'].transform('count')

In [24]:
data['user_refurl_share'] = data['user_cnt']/ data['ref_url_cnt']
data['user_pubid_share'] = data['user_cnt']/ data['pubId_cnt']
data['user_compid_share'] = data['user_cnt']/data['compId_cnt']

In [25]:
data.drop(['ID','ClickDate'], axis=1, inplace=True)
train = data[:len_train]
test = data[len_train:]


In [26]:
del data
gc.collect()

70

In [44]:
def runLgb(train_X, train_y, valid_X, valid_y, feature_names=None, seed_val=0, num_boost_round=300):
    params = {}
    params['max_bin'] = 10
    params['learning_rate'] = 0.0021 # shrinkage_rate
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'regression'
    params['metric'] = 'rmse'          # or 'mae'
    params['sub_feature'] = 0.5      # feature_fraction 
    params['bagging_fraction'] = 0.85 # sub_row
    params['bagging_freq'] = 40
    params['num_leaves'] = 512        # num_leaf
    params['min_data'] = 500         # min_data_in_leaf
    params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf

   
    #print (train_X.shape, train_y.shape, valid_X.shape, valid_y.shape)
   
    lgb_train = lgbm.Dataset(train_X, train_y)
    lgb_valid = lgbm.Dataset(valid_X, valid_y, reference=lgb_train)    
    
   
    watchlist = [lgb_valid ]
    model = lgbm.train(params, lgb_train, num_boost_round, watchlist, early_stopping_rounds=20)
    
    pred_test_y = model.predict(valid_X)
    del train_X, valid_X, train_y, valid_y,lgb_train,lgb_valid
    gc.collect()
    return pred_test_y, model

In [ ]:
cv_scores = []
#i = 1
y_preds = pd.DataFrame()
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=32)
for dev_index, val_index in kf.split(range(train.shape[0])):
        train_X, valid_X = train.iloc[dev_index,:], train.iloc[val_index,:]
        train_y, valid_y = y_train[dev_index], y_train[val_index]
        #print (train_X.shape, train_y.shape, valid_X.shape, valid_y.shape)
        preds, model = runLgb(train_X, train_y, valid_X, valid_y)
        cv_scores.append(mean_squared_error(valid_y, preds))
        #y_preds['gbm'+ str(i)] = model.predict(test, num_iteration=model.best_iteration)
        #i += 1
        print(cv_scores)
        

[1]	valid_0's l2: 34.0453
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's l2: 34.0453
[3]	valid_0's l2: 34.0453
[4]	valid_0's l2: 34.0452
[5]	valid_0's l2: 34.0452
[6]	valid_0's l2: 34.0452
[7]	valid_0's l2: 34.0452
[8]	valid_0's l2: 34.0451
[9]	valid_0's l2: 34.0451
[10]	valid_0's l2: 34.0451
[11]	valid_0's l2: 34.0451
[12]	valid_0's l2: 34.045
[13]	valid_0's l2: 34.045
[14]	valid_0's l2: 34.045
[15]	valid_0's l2: 34.045
[16]	valid_0's l2: 34.045
[17]	valid_0's l2: 34.045
[18]	valid_0's l2: 34.0449
[19]	valid_0's l2: 34.0449
[20]	valid_0's l2: 34.0449
[21]	valid_0's l2: 34.0449
[22]	valid_0's l2: 34.0448
[23]	valid_0's l2: 34.0448
[24]	valid_0's l2: 34.0448
[25]	valid_0's l2: 34.0448
[26]	valid_0's l2: 34.0447
[27]	valid_0's l2: 34.0447
[28]	valid_0's l2: 34.0447
[29]	valid_0's l2: 34.0447
[30]	valid_0's l2: 34.0447
[31]	valid_0's l2: 34.0446
[32]	valid_0's l2: 34.0446
[33]	valid_0's l2: 34.0446
[34]	valid_0's l2: 34.0446
[35]	valid_0's l2: 34.0446
[36]	val

[298]	valid_0's l2: 34.0409
[299]	valid_0's l2: 34.0409
[300]	valid_0's l2: 34.0409


In [ ]:
del train_X, valid_X, train_y, valid_y,
gc.collect()

In [ ]:
y_pred = mo.predict(test, num_iteration=gbm.best_iteration)

In [ ]:
sub = pd.DataFrame({'ID': test_Id, 'ConversionPayOut' : y_pred })
sub = sub[['ID','ConversionPayOut'] ]

In [ ]:
su.head(5)

In [ ]:
sub.to_csv('lgb_v12.csv', index=False)